In [8]:
import numpy as np
import pandas as pd

from PIL import Image 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
import re
import cv2

import skimage
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog

from scipy.sparse import csr_matrix
import h5py

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # Pour les performances de classification

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC

from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin

import warnings
warnings.filterwarnings("ignore")

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from joblib import parallel_backend
from joblib import Parallel, delayed
#from sklearnex import patch_sklearn 
#patch_sklearn()



In [2]:
# Charger un tableau NumPy à partir du fichier HDF5
with h5py.File('data_0_comp_100.h5', 'r') as hf:
    X_train_img_id = hf['image_ids'][:]
    X_train_product_id = hf['product_ids'][:]
    #X_train_gray = hf['X_train_gray'][:]
    X_train_hog = hf['X_train_hog'][:]
    #X_train_hsv = hf['X_train_hsv'][:]
    

In [3]:

# flatten l'image
X_train_var = np.array([image.flatten() for image in X_train_hog])

# lecture du fichier texte
df_X_train_txt = pd.read_csv('text_lemm.csv', delimiter=',')

#on filtre sur les images qui ont bien été traitées
df_features = df_X_train_txt[df_X_train_txt['imageid'].isin(X_train_img_id)]

y = df_features['prdtypecode']

# Séparer le jeu de données en données d'entraînement et données test 
X_train, X_test, X_img_train, X_img_test, y_train, y_test = train_test_split(df_features['Text'], X_train_var, y, test_size=0.2, random_state = 30)

# Vecteur TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# on combine nos deux features
X_train_combined = np.concatenate((X_train_tfidf.toarray(), X_img_train), axis=1)
X_test_combined = np.concatenate((X_test_tfidf.toarray(), X_img_test), axis=1)


In [4]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_combined = sc.fit_transform(X_train_combined)
X_test_combined = sc.transform(X_test_combined)


In [ ]:
%%timeit

# Specify the backend as 'multiprocessing' or 'threading'
# 'multiprocessing' is recommended for CPU-bound tasks, while 'threading' is more suitable for IO-bound tasks

#with parallel_backend('multiprocessing'):

model = LinearSVC()

# Train the model
model.fit(X_train_combined, y_train)


In [6]:
def train_model(X, y):
    model = LinearSVC()
    model.fit(X, y)
    return model
    

In [ ]:
n_jobs = -1  # Utilise tous les cœurs disponibles
models = Parallel(n_jobs=n_jobs)(delayed(train_model)(X_train_combined, y_train) for _ in range(10))


In [ ]:
print(model.score(X_test_combined, y_test))

# Prédiction sur l'échantillon de test
test_pred = model.predict(X_test_combined)

print("Précision de la prédiction:", accuracy_score(y_test, test_pred)*100, '%')

print("Evaluation détaillée de la Classification par RDF :\n \n" ,
      (classification_report(y_test, test_pred)))
